In [1]:
# to set up new kernel for notebook:
# https://stackoverflow.com/questions/58119823/jupyter-notebooks-in-vscode-does-not-use-active-virtual-environment

# python3 -m ipykernel install --user --name=projectnam


# get dataframes of flood levels
from helper_methods import *
xml_root = get_xml_root('https://water.weather.gov/ahps2/hydrograph_to_xml.php?gage=rmdv2&output=xml')
observed_data = get_flood_data(xml_root, 'observed')
forecast_data = get_flood_data(xml_root, 'forecast')
forecast_data = bridge_to_fore(observed_data, forecast_data)

print(observed_data)
print(bridge_data)
print(forecast_data)


                   Time Level
0   2020-10-15 18:45:00  5.73
1   2020-10-15 19:00:00  5.72
2   2020-10-15 19:15:00  5.72
3   2020-10-15 19:30:00  5.71
4   2020-10-15 19:45:00  5.70
..                  ...   ...
183 2020-10-17 16:30:00  5.65
184 2020-10-17 16:45:00  5.64
185 2020-10-17 17:00:00  5.64
186 2020-10-17 17:15:00  5.64
187 2020-10-17 17:30:00  5.64

[188 rows x 2 columns]
       Level                    Time
0    5.63077  2020-10-17 05:45:00 PM
1    5.62154  2020-10-17 06:00:00 PM
2    5.61231  2020-10-17 06:15:00 PM
3    5.60308  2020-10-17 06:30:00 PM
4    5.59385  2020-10-17 06:45:00 PM
..       ...                     ...
236      5.0  2020-10-20 08:45:00 AM
237      5.0  2020-10-20 09:00:00 AM
238      5.0  2020-10-20 09:15:00 AM
239      5.0  2020-10-20 09:30:00 AM
240      5.0  2020-10-20 09:45:00 AM

[266 rows x 2 columns]
                       Time Level
0    2020-10-18 12:00:00 AM   5.4
1    2020-10-18 12:15:00 AM   5.4
2    2020-10-18 12:30:00 AM   5.4
3    2020-10

In [ ]:
import dash
import dash_html_components as html
import plotly
from dash.dependencies import Input, Output
# MY IMPORTS
from helper_methods import *
from layout import *



##################################################################################
##################################_LAYOUT_########################################
##################################################################################
# Create App

app = dash.Dash(__name__)
app.layout = html.Div(
    html.Div([
        html.H4('James River Flood Forecast'),
        html.Div(id='live-update-text'),
        create_flood_graph(
            observed_df=get_observed_data(),
            forecast_df=get_forecast_data(),
        ),
        create_flood_interval(15)
    ])
)

if __name__ == '__main__':
    app.run_server(debug=True,port=6969)
    app.run_server(debug=True, dev_tools_silence_routes_logging = False)
    app.run_server(debug=True)


In [28]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(1, 1)

critical_zones = [{'zone': 2,'level': 9}, {'zone': 3,'level': 11}, {'zone': 4,'level': 12}, {'zone': 5,'level': 15}, {'zone': 6,'level': 22}] # excluding the record line, would be better as a line plot b/c it is a decimal number 
x_range = range(50)
x_values = []
y_values = []

critical_zones_index = 0
for zone in critical_zones:
    for x in x_range:
        for z in range(zone['zone']):
            x_values.append(x)
            y_values.append(zone['level'] + 1) # the plus on is because plotly's histogram colors the cell below the y-value
            lvl = zone['level'] + 1
        while lvl != critical_zones[critical_zones_index]['level']:
            lvl + 1
            for k in range(zone['zone']):
                x_values.append(x)
                y_values.append(lvl + 1)
    critical_zones_index += 1 

# xvalues = []
# for val in xvalues:
#     for i in range()
# print(xvalues)
fig.add_trace(go.Histogram2d(
    x = x_values,
    y = y_values,
    coloraxis = "coloraxis",
    xbins = {'start':1, 'size':1},
    ybins = {'start':1, 'size':1}, 
))

fig.show()


MemoryError: 

In [2]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=[1, 4], 
    y=[9, 9],
    fill=None,
    mode='lines',
    line_color='#ff0000',
    ))
fig.add_trace(go.Scatter(
    x=[1, 4],
    y=[12, 12],
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='#ff0000'))
fig.add_trace(go.Scatter(
    x=[1, 4],
    y=[15, 15],
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='#cc0000'))
fig.add_trace(go.Scatter(
    x=[1, 4],
    y=[22, 22],
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='#990000'))

fig.show()